EvoGGen - Reproducing Behaviour via Evolutionary Grammar-Based Input Generation
===

**EvoGGen**, an acronym for Evolutionary Grammar-Based Input Generation, is an advanced extension of _EvoGFuzz_. It deploys evolutionary optimization strategies to systematically generate inputs that reproduce particular behaviours. The primary objective of EvoGGen is to learn a probabilistic grammar that encapsulates the characteristics of a user-specified behaviour. Consequently, it can generate new, unseen inputs that can trigger this defined behaviour, thereby expanding the boundaries of behaviour-oriented input generation.

Contrasting with EvoGFuzz: While the principal objective of EvoGFuzz is the detection of inputs inducing failures, EvoGGen aims for a different goal. It seeks to not only reproduce a predefined behaviour but also to generalize it, thereby broadening the range of inputs capable of triggering such behaviour.

In [1]:
import math

def calculator(inp: str) -> float:
    """
        A simple calculator function that can evaluate arithmetic expressions 
        and perform basic trigonometric functions and square root calculations.
    """
    return eval(
        str(inp), {"sqrt": math.sqrt, "sin": math.sin, "cos": math.cos, "tan": math.tan}
    )

In [2]:
# Make sure you use the OracleResult from the evogfuzz library
from debugging_framework.input.oracle import OracleResult

def oracle(inp: str):
    """
    This function serves as an oracle or intermediary that catches and handles exceptions 
    generated by the 'calculator' function. The oracle function is used in the context of fuzz testing.
    It aims to determine whether an input triggers a bug in the 'calculator' function.

    Args:
        inp (str): The input string to be passed to the 'calculator' function.

    Returns:
        OracleResult: An enumerated type 'OracleResult' indicating the outcome of the function execution.
            - OracleResult.PASSING: Returned if the calculator function executes without any exception or only with CalculatorSyntaxError
            - OracleResult.FAILING: Returned if the calculator function raises a ValueError exception, indicating a potential bug.
    """
    try:
        calculator(inp)
    except ValueError as e:
        return OracleResult.FAILING, e
    
    return OracleResult.PASSING, None

In [3]:
initial_inputs = ['sqrt(-1)', 'cos(912)', 'tan(4)']

for inp in initial_inputs:
    print(inp.ljust(20), oracle(inp))

sqrt(-1)             (<OracleResult.FAILING: 'FAILING'>, ValueError('math domain error'))
cos(912)             (<OracleResult.PASSING: 'PASSING'>, None)
tan(4)               (<OracleResult.PASSING: 'PASSING'>, None)


In [4]:
from debugging_framework.types import Grammar
from debugging_framework.fuzzingbook.grammar import is_valid_grammar

CALCGRAMMAR: Grammar = {
    "<start>":
        ["<function>(<term>)"],

    "<function>":
        ["sqrt", "tan", "cos", "sin"],
    
    "<term>": ["-<value>", "<value>"], 
    
    "<value>":
        ["<integer>.<integer>",
         "<integer>"],

    "<integer>":
        ["<digit><integer>", "<digit>"],

    "<digit>":
        ["1", "2", "3", "4", "5", "6", "7", "8", "9"]
}
    
assert is_valid_grammar(CALCGRAMMAR)

In [5]:
from evogfuzz.evogfuzz_class import EvoGGen

evoggen = EvoGGen(
    grammar=CALCGRAMMAR,
    inputs=['sqrt(-1.2)'],
    oracle=oracle,
    iterations=20
)

In [6]:
failure_probabilistic_grammar, found_failing_inputs = evoggen.optimize()

Let us look at the learned failure-inducing probabilistic grammar:

In [7]:
from pprint import pprint
pprint(failure_probabilistic_grammar)

{'<digit>': [('1', {'prob': 0.10844306738962045}),
             ('2', {'prob': 0.10224632068164213}),
             ('3', {'prob': 0.10999225406661503}),
             ('4', {'prob': 0.10379550735863671}),
             ('5', {'prob': 0.09914794732765299}),
             ('6', {'prob': 0.12316034082106894}),
             ('7', {'prob': 0.1262587141750581}),
             ('8', {'prob': 0.12161115414407436}),
             ('9', {'prob': 0.10534469403563129})],
 '<function>': [('sqrt', {'prob': 1.0}),
                ('tan', {'prob': 0.0}),
                ('cos', {'prob': 0.0}),
                ('sin', {'prob': 0.0})],
 '<integer>': [('<digit><integer>', {'prob': 0.5507358636715725}),
               ('<digit>', {'prob': 0.4492641363284276})],
 '<start>': [('<function>(<term>)', {'prob': None})],
 '<term>': [('-<value>', {'prob': 1.0}), ('<value>', {'prob': 0.0})],
 '<value>': [('<integer>.<integer>', {'prob': 0.6909620991253644}),
             ('<integer>', {'prob': 0.30903790087463556})]}


While learning the failure-inducing-grammar, we also found the following failing inputs:

In [8]:
# we only display the first 10 failing inputs
for inp in list(found_failing_inputs)[:10]:
    print(str(inp).ljust(20), inp.oracle)

sqrt(-8.55)          FAILING
sqrt(-79)            FAILING
sqrt(-9.6)           FAILING
sqrt(-91)            FAILING
sqrt(-2.9)           FAILING
sqrt(-37.9)          FAILING
sqrt(-955.41117)     FAILING
sqrt(-12.6)          FAILING
sqrt(-98)            FAILING
sqrt(-1.6)           FAILING


Now, we can use a **proabilistic grammar-based fuzzer** to generate more failing inputs with the `failure_probabilistic_grammar`:

In [9]:
from debugging_framework.fuzzingbook.probalistic_fuzzer import ProbabilisticGrammarFuzzer

prob_fuzzer = ProbabilisticGrammarFuzzer(failure_probabilistic_grammar)

for _ in range(100):
    inp = prob_fuzzer.fuzz()
    print(inp.ljust(20), oracle(inp))

sqrt(-188.5)         (<OracleResult.FAILING: 'FAILING'>, ValueError('math domain error'))
sqrt(-91)            (<OracleResult.FAILING: 'FAILING'>, ValueError('math domain error'))
sqrt(-7.91)          (<OracleResult.FAILING: 'FAILING'>, ValueError('math domain error'))
sqrt(-738)           (<OracleResult.FAILING: 'FAILING'>, ValueError('math domain error'))
sqrt(-9.1)           (<OracleResult.FAILING: 'FAILING'>, ValueError('math domain error'))
sqrt(-568.81659)     (<OracleResult.FAILING: 'FAILING'>, ValueError('math domain error'))
sqrt(-742.2212)      (<OracleResult.FAILING: 'FAILING'>, ValueError('math domain error'))
sqrt(-8.84)          (<OracleResult.FAILING: 'FAILING'>, ValueError('math domain error'))
sqrt(-742.2)         (<OracleResult.FAILING: 'FAILING'>, ValueError('math domain error'))
sqrt(-9.8742)        (<OracleResult.FAILING: 'FAILING'>, ValueError('math domain error'))
sqrt(-7.8)           (<OracleResult.FAILING: 'FAILING'>, ValueError('math domain error'))
sqrt(-5.9)